In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20
CLASSES_LIST = ["smoke", "shoot_gun", "run", "hit"]
convlstm_model_h5 = load_model("./convlstm_model.h5", compile=False)
convlstm_model_h5.compile(
    optimizer="rmsprop",
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
)

def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    video_reader = cv2.VideoCapture(video_file_path)
    frames_list = []
    predicted_class_name = ""
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)
    predicted_labels_probabilities = convlstm_model_h5.predict(
        np.expand_dims(frames_list, axis=0)
    )[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    print(
        f"Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}"
    )
    video_reader.release()


In [2]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO("yolov8n.pt")
license_plate_detector = YOLO("license_plate_detector.pt")


Using CPU. Note: This module is much faster with a GPU.


In [3]:
# Objects unique IDs array
objects = []

# Initialize the Kalman Filter for each object
for object in objects:
    object.initialize_kalman_filter()

# Process video frames
for frame in video_frames:
    # Detect objects in the frame
    detected_objects = detect_objects(frame)

    # Update tracked objects with new detections
    for object in objects:
        object.predict()  # Predict the object's new position
        object.update_association(detected_objects)  # Associate the object with detected objects
        
    # Create new tracking objects for unmatched detections
    new_objects = create_new_objects(detected_objects, objects)

    # Add new objects to the list of tracked objects
    objects.extend(new_objects)

    # Remove lost objects
    objects = remove_inactive_objects(objects)

    # Post process tracked objects
    post_process_objects(objects)

    # Draw bounding boxes on the frame for visualization
    draw_objects_on_frame(frame, objects)

    #save the processed frame
    display_frame(frame)


NameError: name 'video_frames' is not defined

In [4]:
def track_number_plate(video_file_path, srch_num=None):
    # load video
    cap = cv2.VideoCapture(video_file_path)

    vehicles = [2, 3, 5, 7]

    # read frames
    frame_nmr = -1
    ret = True
    while ret:
        frame_nmr += 1
        ret, frame = cap.read()
        if ret:
            results[frame_nmr] = {}
            # detect vehicles
            print("1111111111111")
            detections = coco_model(frame)[0]
            detections_ = []
            for detection in detections.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = detection
                if int(class_id) in vehicles:
                    detections_.append([x1, y1, x2, y2, score])
            print("222222222222222")
            # track vehicles
            track_ids = mot_tracker.update(np.asarray(detections_))

            # detect license plates
            license_plates = license_plate_detector(frame)[0]
            for license_plate in license_plates.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = license_plate

                # assign license plate to car
                xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

                if car_id != -1:
                    # crop license plate
                    license_plate_crop = frame[int(y1) : int(y2), int(x1) : int(x2), :]

                    # process license plate
                    license_plate_crop_gray = cv2.cvtColor(
                        license_plate_crop, cv2.COLOR_BGR2GRAY
                    )
                    _, license_plate_crop_thresh = cv2.threshold(
                        license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV
                    )

                    # read license plate number
                    license_plate_text, license_plate_text_score = read_license_plate(
                        license_plate_crop_thresh
                    )

                    if license_plate_text is not None:
                        # print(
                        #     "++++++++++++++++++++++++++++++++++++++++++++++   ",
                        #     license_plate_text
                        #     + "  ++++++++++++++++++++++++++++++++++++++++++++++",
                        # )
                        results[frame_nmr][car_id] = {
                            "car": {"bbox": [xcar1, ycar1, xcar2, ycar2]},
                            "license_plate": {
                                "bbox": [x1, y1, x2, y2],
                                "text": license_plate_text,
                                "bbox_score": score,
                                "text_score": license_plate_text_score,
                            },
                        }

    # write results
    write_csv(results, "test.csv")

In [ ]:
track_number_plate("test_2.mp4")

In [ ]:
import cv2
import os
import shutil
import threading

def prediction_engine():
    while True:
        dir = os.listdir("D:/Project_v0.0.0/live_footages")
        for i in dir:
            if i and i.startswith("f"):
                path = os.path.join("D:/Project_v0.0.0/live_footages/", i)
                prediction_thread = threading.Thread(target=predict_single_action,args=(path,20))
                # tracking_thread = threading.Thread(target=track_number_plate,args=(path,"okkokk"))
                print(path)
                prediction_thread.start()
                # tracking_thread.start()
                prediction_thread.join()
                # tracking_thread.join()
                # tracking_thread.join()
                if not prediction_thread.is_alive() :
                    shutil.move(path, "D:/Project_v0.0.0/upload_engine")
                    print("file moved")

def main():
    thread_1 = threading.Thread(target=prediction_engine)
    thread_1.start()
    cap = cv2.VideoCapture("test.mp4")

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))

    fourcc = cv2.VideoWriter_fourcc(*"H264")  # Codec for MP4 format
    stream_1 = None
    stream_2 = None
    s1_fileName = None
    s2_fileName = None
    recording_timer = 0
    file_no = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame.")
            if len(os.listdir("D:/Project_v0.0.0/live_footages")) > 0:
                for i in os.listdir("D:/Project_v0.0.0/live_footages"):
                    if i and not i.startswith("f"):
                        os.rename(
                            f"D:/Project_v0.0.0/live_footages/{i}",
                            f"D:/Project_v0.0.0/live_footages/f_{i}",
                        )
            thread_1.join()
            break
        if recording_timer == fps * 0:
            stream_1 = cv2.VideoWriter(
                "D:/Project_v0.0.0/live_footages/output_{}_s1.mp4".format(file_no),
                fourcc,
                fps,
                (frame_width, frame_height),
            )

            s1_fileName = "output_{}_s1.mp4".format(file_no)
            file_no += 1
        if recording_timer == fps * 7:
            stream_2 = cv2.VideoWriter(
                "D:/Project_v0.0.0/live_footages/output_{}_s2.mp4".format(file_no),
                fourcc,
                fps,
                (frame_width, frame_height),
            )
            s2_fileName = "output_{}_s2.mp4".format(file_no)
            file_no += 1

        if recording_timer >= fps * 0 and recording_timer <= fps * 10:
            stream_1.write(frame)
            if recording_timer == fps * 10:
                stream_1.release()
                os.rename(
                    f"D:/Project_v0.0.0/live_footages/{s1_fileName}",
                    f"D:/Project_v0.0.0/live_footages/f_{s1_fileName}",
                )
        if recording_timer >= fps * 7 and recording_timer <= fps * 17:
            stream_2.write(frame)
            if recording_timer == fps * 17:
                stream_2.release()
                os.rename(
                    f"D:/Project_v0.0.0/live_footages/{s2_fileName}",
                    f"D:/Project_v0.0.0/live_footages/f_{s2_fileName}",
                )

        if recording_timer == fps * 17:
            recording_timer = 0
        else:
            recording_timer += 1

        cv2.imshow("Recording", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    if stream_1 is not None:
        stream_1.release()
    if stream_2 is not None:
        stream_2.release()

    cap.release()
    cv2.destroyAllWindows()
    # if os.listdir("D:/Project_v0.0.0/live_footages") == 0:
    thread_1.join()


if __name__ == "__main__":
    main()